# Regresión Logística

Como estamos hablando de Regresión Lógística ya sabemos que vamos a ver si supera la probabilidad de  que sea en la parte superior de la función (con forma de S) o la parte inferior y aunque podemos combinar varias regresiones para mejorar el resultado, ví que sklearn tiene dos enfoques uno que permite con una sola función detectar múltiples clases o solo trabajar de manera Binarizada, por eso en este trabajo solo implementé una, porque ya sabía que el modelo que mejor se iba a adaptar era la red neuronal CNN; al principo solo me daba 10%, pero utilziando HOG para detectar texturas y formas; y fourier logré aumentar en el mejor caso hasta 40%

In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from skimage.feature import hog

# Ruta a la carpeta principal que contiene las subcarpetas
ruta_carpeta_general = 'train/'

# Listas para almacenar las imágenes y etiquetas
imagenes = []
etiquetas = []

# Función para extraer características HOG
def extraer_caracteristicas_hog(imagen):
    return hog(imagen, pixels_per_cell=(16, 16), cells_per_block=(2, 2), orientations=9)

# Función para extraer características Fourier
def extraer_caracteristicas_fourier(imagen):
    f_transformada = np.fft.fft2(imagen)
    f_shifted = np.fft.fftshift(f_transformada)  
    magnitud_espectro = np.abs(f_shifted)        
    # Redimensionamos para reducir tamaño y convertimos en vector
    magnitud_espectro = cv2.resize(magnitud_espectro, (32, 32)).flatten()
    return magnitud_espectro

# Recorrer las subcarpetas
for subcarpeta in os.listdir(ruta_carpeta_general):
    ruta_subcarpeta = os.path.join(ruta_carpeta_general, subcarpeta)
    if os.path.isdir(ruta_subcarpeta):
        for archivo in os.listdir(ruta_subcarpeta):
            ruta_imagen = os.path.join(ruta_subcarpeta, archivo)
            # Cargar la imagen en escala de grises y redimensionarla
            imagen = cv2.imread(ruta_imagen, cv2.IMREAD_GRAYSCALE)
            imagen = cv2.resize(imagen, (128, 128))
            # Extraer características HOG y Fourier
            caracteristicas_hog = extraer_caracteristicas_hog(imagen)
            caracteristicas_fourier = extraer_caracteristicas_fourier(imagen)
            # Combinar características de HOG y Fourier
            caracteristicas_combinadas = np.concatenate([caracteristicas_hog, caracteristicas_fourier])
            imagenes.append(caracteristicas_combinadas)
            etiquetas.append(subcarpeta)

# Convertir listas a arrays de NumPy
X = np.array(imagenes)
y = np.array(etiquetas)

# Escalado de características antes de PCA
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Aplicar PCA para reducir la dimensionalidad
pca = PCA(n_components=50)
X_pca = pca.fit_transform(X)

# Dividir los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.1, random_state=42)

# Crear y entrenar el modelo de regresión logística
modelo = LogisticRegression(max_iter=2000, solver='liblinear', C=0.1)
modelo.fit(X_train, y_train)

# Evaluar el modelo
y_pred = modelo.predict(X_test)
precision = accuracy_score(y_test, y_pred)
print(f'Precisión del modelo: {precision * 100:.2f}%')

Precisión del modelo: 40.11%


In [ ]:
import pandas as pd

# Ruta a la carpeta de prueba (test)
ruta_carpeta_test = 'test/'

# Listas para almacenar las imágenes y nombres de archivo
imagenes_test = []
nombres_archivos = []

# Función para extraer características HOG (misma que en el entrenamiento)
def extraer_caracteristicas_hog(imagen):
    return hog(imagen, pixels_per_cell=(16, 16), cells_per_block=(2, 2), orientations=9)

# Función para extraer características Fourier (misma que en el entrenamiento)
def extraer_caracteristicas_fourier(imagen):
    f_transformada = np.fft.fft2(imagen)
    f_shifted = np.fft.fftshift(f_transformada)
    magnitud_espectro = np.abs(f_shifted)
    magnitud_espectro = cv2.resize(magnitud_espectro, (32, 32)).flatten()
    return magnitud_espectro

# Recorrer la carpeta de prueba y ordenar los archivos alfabéticamente
for archivo in sorted(os.listdir(ruta_carpeta_test)):
    ruta_imagen = os.path.join(ruta_carpeta_test, archivo)
    # Cargar la imagen en escala de grises y redimensionarla al mismo tamaño que las imágenes de entrenamiento
    imagen = cv2.imread(ruta_imagen, cv2.IMREAD_GRAYSCALE)
    if imagen is not None:
        imagen = cv2.resize(imagen, (128, 128))  
        # Extraer características HOG y Fourier
        imagen_hog = extraer_caracteristicas_hog(imagen)
        imagen_fourier = extraer_caracteristicas_fourier(imagen)
        # Combinar características de HOG y Fourier
        imagen_caracteristicas = np.concatenate([imagen_hog, imagen_fourier])
        imagenes_test.append(imagen_caracteristicas)
        nombres_archivos.append(archivo)

# Convertir la lista de características combinadas a un array de NumPy
X_test = np.array(imagenes_test)

# Escalar las características usando el mismo escalador del entrenamiento
X_test = scaler.transform(X_test)

# Reducir dimensiones con PCA
X_test_pca = pca.transform(X_test)

# Hacer predicciones con el modelo entrenado
predicciones = modelo.predict(X_test_pca)

# Crear un DataFrame para almacenar los resultados
resultados = pd.DataFrame({
    'file': nombres_archivos,   
    'label': predicciones       
})

# Guardar los resultados en un archivo CSV
resultados.to_csv('Relog.csv', index=False)

# CNN
En este proceso tuve dos inconvenientes, el primero es las librerías no me estaban funcionando y el segundo es que se me acabaron los creditos de google colab(con kaggle duré 7 horas en una sesión y no me llegó a dar resultado); pero este fue uno de los modelos que mejor se adaptó, y con lo poco que logré probar en kaggle aumentando la resolución de la imagen, con más epocas ya el acuracy en test era del 82%, así que si pudiese terminar la compilación lo más seguro es que  mejorara este resultado

In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Configuración inicial
img_height, img_width = 32, 32  
ruta_carpeta_general = 'train/'

# Listas para almacenar imágenes y etiquetas
imagenes = []
etiquetas = []

# Cargar imágenes y etiquetas desde las subcarpetas
for subcarpeta in os.listdir(ruta_carpeta_general):
    ruta_subcarpeta = os.path.join(ruta_carpeta_general, subcarpeta)
    if os.path.isdir(ruta_subcarpeta):
        for archivo in os.listdir(ruta_subcarpeta):
            ruta_imagen = os.path.join(ruta_subcarpeta, archivo)
            imagen = cv2.imread(ruta_imagen)
            imagen = cv2.resize(imagen, (img_height, img_width))
            imagenes.append(imagen)
            etiquetas.append(subcarpeta)

# Convertir listas a arrays y normalizar
X = np.array(imagenes) / 255.0  
y = np.array(etiquetas)

# Binarización de las etiquetas
lb = LabelBinarizer()
y = lb.fit_transform(y)
num_classes = len(lb.classes_)

# División de datos en entrenamiento y prueba
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Aumento de datos
datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)
datagen.fit(X_train)

# Definir el modelo Sequential con capas convolucionales
model = tf.keras.Sequential([
    layers.Conv2D(32, (3, 3), padding='same', input_shape=(img_height, img_width, 3)),
    layers.BatchNormalization(),
    layers.LeakyReLU(alpha=0.01),
    layers.Conv2D(32, (3, 3), padding='same'),
    layers.BatchNormalization(),
    layers.LeakyReLU(alpha=0.01),
    layers.MaxPooling2D(pool_size=(2, 2)),

    layers.Conv2D(64, (3, 3), padding='same'),
    layers.BatchNormalization(),
    layers.LeakyReLU(alpha=0.01),
    layers.Conv2D(64, (3, 3), padding='same'),
    layers.BatchNormalization(),
    layers.LeakyReLU(alpha=0.01),
    layers.MaxPooling2D(pool_size=(2, 2)),

    layers.Conv2D(128, (3, 3), padding='same'),
    layers.BatchNormalization(),
    layers.LeakyReLU(alpha=0.01),
    layers.Conv2D(128, (3, 3), padding='same'),
    layers.BatchNormalization(),
    layers.LeakyReLU(alpha=0.01),
    layers.MaxPooling2D(pool_size=(2, 2)),

    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.5),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(num_classes, activation='softmax')
])

# Compilar el modelo
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Callbacks para evitar sobreajuste y reducir la tasa de aprendizaje
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-5)

# Entrenar el modelo
history = model.fit(
    datagen.flow(X_train, y_train, batch_size=32),
    epochs=50,
    validation_data=(X_val, y_val),
    callbacks=[early_stopping, reduce_lr],
    verbose=1
)

# Evaluación del modelo en el conjunto de prueba
y_pred = model.predict(X_val)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_val, axis=1)

# Imprimir la matriz de confusión y el reporte de clasificación
print("Reporte de clasificación:\n", classification_report(y_true_classes, y_pred_classes, target_names=lb.classes_))
print("Matriz de confusión:\n", confusion_matrix(y_true_classes, y_pred_classes))


C:\Users\jposa\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\jposa\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\activations\leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(
C:\Users\jposa\AppData\Roaming\Python\Python312\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50
136/136 ━━━━━━━━━━━━━━━━━━━━ 35s 136ms/step - accuracy: 0.1280 - loss: 2.9036 - val_accuracy: 0.1128 - val_loss: 3.4537 - learning_rate: 0.0010
Epoch 2/50
136/136 ━━━━━━━━━━━━━━━━━━━━ 17s 124ms/step - accuracy: 0.2737 - loss: 2.1679 - val_accuracy: 0.1155 - val_loss: 4.1653 - learning_rate: 0.0010
Epoch 3/50
136/136 ━━━━━━━━━━━━━━━━━━━━ 17s 125ms/step - accuracy: 0.3247 - loss: 1.9202 - val_accuracy: 0.2274 - val_loss: 2.7129 - learning_rate: 0.0010
Epoch 4/50
136/136 ━━━━━━━━━━━━━━━━━━━━ 17s 125ms/step - accuracy: 0.3697 - loss: 1.7982 - val_accuracy: 0.3808 - val_loss: 1.6756 - learning_rate: 0.0010
Epoch 5/50
136/136 ━━━━━━━━━━━━━━━━━━━━ 17s 127ms/step - accuracy: 0.3935 - loss: 1.6820 - val_accuracy: 0.2939 - val_loss: 2.3915 - learning_rate: 0.0010
Epoch 6/50
136/136 ━━━━━━━━━━━━━━━━━━━━ 17s 128ms/step - accuracy: 0.4232 - loss: 1.5666 - val_accuracy: 0.2625 - val_loss: 1.9325 - learning_rate: 0.0010
Epoch 7/50
136/136 ━━━━━━━━━━━━━━━━━━━━ 17s 127ms/step - accuracy: 0.4

In [ ]:
# Importar las bibliotecas necesarias
import pandas as pd

# Ruta de la carpeta de prueba
ruta_carpeta_test = 'test/'

# Listas para almacenar nombres de archivos y predicciones
nombres_archivos = []
predicciones = []

# Cargar imágenes desde la carpeta de prueba
for archivo in os.listdir(ruta_carpeta_test):
    ruta_imagen = os.path.join(ruta_carpeta_test, archivo)
    imagen = cv2.imread(ruta_imagen)
    if imagen is not None:
        imagen = cv2.resize(imagen, (img_height, img_width))
        imagen = np.array(imagen) / 255.0  
        imagen = np.expand_dims(imagen, axis=0)  

        # Realizar la predicción
        prediccion = model.predict(imagen, verbose=0)
        clase_predicha = np.argmax(prediccion, axis=1)[0]  
        predicciones.append(clase_predicha)  
        nombres_archivos.append(archivo)  

# Crear un DataFrame para almacenar los resultados
resultados = pd.DataFrame({
    'file': nombres_archivos,   
    'label': lb.classes_[predicciones]  
})

# Guardar los resultados en un archivo CSV
resultados.to_csv('CNN_test.csv', index=False)

print("Predicciones guardadas en CNN_test.csv")


Predicciones guardadas en CNN_test.csv
